In [13]:
import os
import json
import numpy as np
import pickle
import re
import torch

### Questions

In [2]:
with open('.\\data\\train_balanced_questions.json', 'r') as f:
    train_q = json.load(f)

In [31]:
qids = sorted(list(train_q.keys()))
qids[:5]

['001', '001000003', '001000004', '001000018', '00100002']

In [5]:
train_q['02930152'].keys()

dict_keys(['semantic', 'entailed', 'equivalent', 'question', 'imageId', 'isBalanced', 'groups', 'answer', 'semanticStr', 'annotations', 'types', 'fullAnswer'])

In [4]:
train_q['02930152']

{'semantic': [{'operation': 'select',
   'dependencies': [],
   'argument': 'sky (2486325)'},
  {'operation': 'verify color', 'dependencies': [0], 'argument': 'dark'}],
 'entailed': ['02930160',
  '02930158',
  '02930159',
  '02930154',
  '02930155',
  '02930156',
  '02930153'],
 'equivalent': ['02930152'],
 'question': 'Is the sky dark?',
 'imageId': '2354786',
 'isBalanced': True,
 'groups': {'global': None, 'local': '06-sky_dark'},
 'answer': 'yes',
 'semanticStr': 'select: sky (2486325)->verify color: dark [0]',
 'annotations': {'answer': {},
  'question': {'2': '2486325'},
  'fullAnswer': {'2': '2486325'}},
 'types': {'detailed': 'verifyAttr',
  'semantic': 'attr',
  'structural': 'verify'},
 'fullAnswer': 'Yes, the sky is dark.'}

### Scene Graphs

In [14]:
with open('.\\data\\train_sceneGraphs.json', 'r') as f:
    train_sg = json.load(f)

set()

In [8]:
sg_ids = list(train_sg.keys())
sg_ids[:5]

['2386621', '2373554', '2370799', '2370791', '2370790']

In [9]:
train_sg['2386621'].keys()

dict_keys(['width', 'objects', 'height'])

In [30]:
train_sg['2386621']

{'width': 500,
 'objects': {'681267': {'name': 'banana',
   'h': 34,
   'relations': [{'object': '681262', 'name': 'to the left of'}],
   'w': 64,
   'attributes': ['small', 'yellow'],
   'y': 55,
   'x': 248},
  '681265': {'name': 'spots',
   'h': 16,
   'relations': [],
   'w': 26,
   'attributes': [],
   'y': 92,
   'x': 245},
  '681264': {'name': 'bananas',
   'h': 50,
   'relations': [{'object': '681259', 'name': 'to the left of'}],
   'w': 49,
   'attributes': ['small', 'yellow'],
   'y': 32,
   'x': 268},
  '681263': {'name': 'picnic',
   'h': 374,
   'relations': [],
   'w': 499,
   'attributes': ['delicious'],
   'y': 0,
   'x': 0},
  '681262': {'name': 'straw',
   'h': 95,
   'relations': [{'object': '681268', 'name': 'to the right of'},
    {'object': '681267', 'name': 'to the right of'},
    {'object': '681253', 'name': 'to the right of'}],
   'w': 15,
   'attributes': ['white', 'plastic'],
   'y': 55,
   'x': 402},
  '681261': {'name': 'meat',
   'h': 27,
   'relations': [

In [13]:
train_sg['2386621']['objects']

{'681267': {'name': 'banana',
  'h': 34,
  'relations': [{'object': '681262', 'name': 'to the left of'}],
  'w': 64,
  'attributes': ['small', 'yellow'],
  'y': 55,
  'x': 248},
 '681265': {'name': 'spots',
  'h': 16,
  'relations': [],
  'w': 26,
  'attributes': [],
  'y': 92,
  'x': 245},
 '681264': {'name': 'bananas',
  'h': 50,
  'relations': [{'object': '681259', 'name': 'to the left of'}],
  'w': 49,
  'attributes': ['small', 'yellow'],
  'y': 32,
  'x': 268},
 '681263': {'name': 'picnic',
  'h': 374,
  'relations': [],
  'w': 499,
  'attributes': ['delicious'],
  'y': 0,
  'x': 0},
 '681262': {'name': 'straw',
  'h': 95,
  'relations': [{'object': '681268', 'name': 'to the right of'},
   {'object': '681267', 'name': 'to the right of'},
   {'object': '681253', 'name': 'to the right of'}],
  'w': 15,
  'attributes': ['white', 'plastic'],
  'y': 55,
  'x': 402},
 '681261': {'name': 'meat',
  'h': 27,
  'relations': [{'object': '681255', 'name': 'on'},
   {'object': '681255', 'name'

### Find Max Question Length (30)

In [25]:
def tokenize(sentence):
    # punctuation should be separated from the words
    s = re.sub('([.,;:!?()])', r' \1 ', sentence)
    s = re.sub('\s{2,}', ' ', s)

    # tokenize
    split = s.split()

    # normalize all words to lowercase
    lower = [w.lower() for w in split]
    return lower

def to_dictionary_indexes(dictionary, sentence):
    """
    Outputs indexes of the dictionary corresponding to the words in the sequence.
    Case insensitive.
    """
    split = tokenize(sentence)
    idxs = np.array([dictionary[w] if w in dictionary else dictionary['UNK'] for w in split])
    return idxs

In [3]:
longest_q = 0
with open('.\\data\\train_balanced_questions.json', 'r') as f:
    train_q = json.load(f)
    for q in train_q:
        question = tokenize(train_q[q]['question'])
        q_len = len(question)
        if q_len > longest_q:
            longest_q = q_len

with open('.\\data\\val_all_questions.json', 'r') as f:
    val_q = json.load(f)
    for q in val_q:
        question = tokenize(val_q[q]['question'])
        q_len = len(question)
        if q_len > longest_q:
            longest_q = q_len

In [4]:
longest_q

30

### Get number of Names, number of Attributes, and maximum number of Attributes for a single object

In [22]:
names = []
attr = []
max_attr = 0

with open('.\\data\\train_sceneGraphs.json', 'r') as f:
    train_sg = json.load(f)
    for sg in train_sg:
        objects = train_sg[sg]['objects']
        for obj in objects:
            names.append(train_sg[sg]['objects'][obj]['name'])
            attr += train_sg[sg]['objects'][obj]['attributes']
            attr_len = len(train_sg[sg]['objects'][obj]['attributes'])
            if attr_len > max_attr:
                max_attr = attr_len

with open('.\\data\\val_sceneGraphs.json', 'r') as f:
    val_sg = json.load(f)
    for sg in val_sg:
        objects = val_sg[sg]['objects']
        for obj in objects:
            names.append(val_sg[sg]['objects'][obj]['name'])
            attr += val_sg[sg]['objects'][obj]['attributes']
            attr_len = len(val_sg[sg]['objects'][obj]['attributes'])
            if attr_len > max_attr:
                max_attr = attr_len

In [28]:
names = set(names)
attr = set(attr)
print(len(names))
print(len(attr))
print(max_attr)

1703
617
36


In [29]:
(4+1704+(618*36))*2

47912

### Make Predictions

In [12]:
preds = []
for ques in qids:
    preds.append({"questionId": ques, "prediction": 'no'})

In [13]:
with open('val_predictions.json', 'w') as f:
    json.dump(preds, f)